<a href="https://colab.research.google.com/github/ashborn8/defectosCafe/blob/main/defectos_del_cafe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Instalar y configurar Kaggle
!pip install kaggle --quiet
from google.colab import files

# Subir el archivo kaggle.json
print("Por favor, sube el archivo 'kaggle.json' descargado desde tu cuenta de Kaggle.")
files.upload()  # Esto abrirá un cuadro para que subas el archivo

# Crear y configurar la carpeta .kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 2. Descargar y extraer el dataset de Kaggle (usando el primer código para la extracción manual)
!kaggle datasets download -d piyushkumar18/dataset-resized-224-x-224 -p ./datasets
!unzip -q ./datasets/dataset-resized-224-x-224.zip -d coffee_images

# Verificar el contenido de las carpetas
!ls coffee_images

# 3. Importar librerías necesarias
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import os

# 4. Preprocesamiento de datos
# Definir funciones de preprocesamiento para las imágenes de los granos de café
IMG_SIZE = 120  # Tamaño de imagen ajustado

def preprocess(image, label):  # imagen y etiqueta
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))  # Redimensionar la imagen
    image = image / 255.0  # Normalizar la imagen entre 0 y 1
    return image, label

# Usando ImageDataGenerator para cargar las imágenes desde las carpetas
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 80% para entrenamiento, 20% para validación
)

# Generador de imágenes de entrenamiento
train_generator = train_datagen.flow_from_directory(
    'coffee_images',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='binary',  # Clasificación binaria (defecto vs no defecto)
    subset='training'  # Subconjunto de entrenamiento
)

# Generador de imágenes de validación
val_generator = train_datagen.flow_from_directory(
    'coffee_images',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='binary',  # Clasificación binaria (defecto vs no defecto)
    subset='validation'  # Subconjunto de validación
)

# 5. Crear el modelo
model = models.Sequential([  # Modelo secuencial
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),  # 32 filtros de 3x3
    layers.MaxPooling2D(2, 2),  # Capa de agrupación
    layers.Conv2D(64, (3, 3), activation='relu'),  # Capa de convolución
    layers.MaxPooling2D(2, 2),  # Capa de agrupación
    layers.Conv2D(128, (3, 3), activation='relu'),  # Capa de convolución
    layers.MaxPooling2D(2, 2),  # Capa de agrupación
    layers.Flatten(),  # Aplanar las imágenes
    layers.Dense(128, activation='relu'),  # Capa densa de conectividad
    layers.Dense(1, activation='sigmoid')  # Capa de salida con activación sigmoide (binaria)
])

# 6. Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 7. Entrenar el modelo
history = model.fit(
    train_generator,
    epochs=10,  # Ajusta el número de épocas si es necesario
    validation_data=val_generator
)

# 8. Evaluar el modelo
loss, accuracy = model.evaluate(val_generator)
print(f'Precisión en el conjunto de validación: {accuracy:.2f}')

# 9. Visualizar los resultados del entrenamiento
# Precisión de entrenamiento y validación
plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'], label='Precisión en Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión en Validación')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.show()

# Pérdida de entrenamiento y validación
plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'], label='Pérdida en Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida en Validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

# 10. Inferencia en una imagen de prueba
import numpy as np
from tensorflow.keras.preprocessing import image

# Definir los nombres de las clases
class_names = ['No defecto', 'Con defecto']

# Función de predicción
def predict_image(image_path):
    # Cargar la imagen
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = tf.expand_dims(img_array, 0)  # Añadir una dimensión para el lote

    # Realizar la predicción
    prediction = model.predict(img_array)
    predicted_class_idx = int(tf.round(prediction[0][0]).numpy())
    confidence = prediction[0][0]

    return predicted_class_idx, confidence

# Ruta de la imagen a predecir
image_path = '/content/coffee_images/test_image.jpg'  # Cambia esto a la ruta de tu imagen
predicted_class_idx, confidence = predict_image(image_path)

# Mostrar los resultados
print(f'Predicción: {class_names[predicted_class_idx]} (Confianza: {confidence:.2f})')

# Visualizar la imagen
img = tf.keras.preprocessing.image.load_img(image_path)
plt.imshow(img)
plt.axis('off')
plt.show()


Por favor, sube el archivo 'kaggle.json' descargado desde tu cuenta de Kaggle.


Saving kaggle.json to kaggle.json
403 - Forbidden - Permission 'datasets.get' was denied
unzip:  cannot find or open ./datasets/dataset-resized-224-x-224.zip, ./datasets/dataset-resized-224-x-224.zip.zip or ./datasets/dataset-resized-224-x-224.zip.ZIP.
ls: cannot access 'coffee_images': No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: 'coffee_images'